In [12]:
from dotenv import load_dotenv
from google.adk.agents import Agent, ParallelAgent

# Load environment variables (GOOGLE_API_KEY)
load_dotenv()

# It's better to define the fetching logic as tools for the agents
# For simplicity in this example, we'll embed the logic in the agent's instruction.
# In a real-world scenario, you would use tools.

# Define the individual agents that will run in parallel
weather_fetcher = Agent(
    name="weather_fetcher",
    model="gemini-2.0-flash-exp",
    instruction="Fetch the weather for the given location and return only the weather report.",
    output_key="weather_data"  # The result will be stored in session.state["weather_data"]
)

news_fetcher = Agent(
    name="news_fetcher",
    model="gemini-2.0-flash-exp",
    instruction="Fetch the top news story for the given topic and return only that story.",
    output_key="news_data"      # The result will be stored in session.state["news_data"]
)

# Create the ParallelAgent to orchestrate the sub-agents
data_gatherer = ParallelAgent(
    name="data_gatherer",
    sub_agents=[
        weather_fetcher,
        news_fetcher
    ]
)

# To run this, you would use the ADK's Runner, which manages state and execution.
# The following is a conceptual example of how it would be invoked.



In [13]:
from google.genai import types
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

# Define variables for session setup
APP_NAME = "data_gatherer"
USER_ID = "user_123"
SESSION_ID = "session_001"

# Session and Runner setup
session_service = InMemorySessionService()
session = await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
)
runner = Runner(
        agent=data_gatherer,
        app_name=APP_NAME,
        session_service=session_service
)

content = types.Content(role='user', parts=[types.Part(text="Get weather for Mountain View and news on technology")])

## Execute the agent
async for event in runner.run_async(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=content
 ):
     if event.is_final_response():
         # Get the session state using the session service
         final_session = await session_service.get_session(
             app_name=APP_NAME,
             user_id=USER_ID,
             session_id=SESSION_ID
         )
         print("\n--- Gathered Data ---")
         print(f"Weather: {final_session.state.get('weather_data')}")
         print(f"News: {final_session.state.get('news_data')}")


--- Gathered Data ---
Weather: ```tool_code
print(google_search.search(queries=["weather report Mountain View"]))
```
News: None

--- Gathered Data ---
Weather: ```tool_code
print(google_search.search(queries=["weather report Mountain View"]))
```
News: Okay, I can get you the weather for Mountain View and the top technology news story. Here's the top tech news story:

**Top Tech News Story:**

*   I am sorry, but due to my current limitations, I cannot access the internet to get the current top tech news story.
